In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

movies.head()

credits.head(1)

movies=movies.merge(credits,on='title')

movies.head(2)

# genres, movie_id, keywords,title,overview, cast, crew

movies=movies[['movie_id','title','genres','keywords','overview','cast','crew']]

movies

import ast

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

movies.dropna(inplace=True)

movies.isnull().sum()

movies.duplicated().sum()

movies['genres'] = movies['genres'].apply(convert)

movies.head()

movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

movies['cast'] = movies['cast'].apply(convert)
movies.head()

movies['cast'] = movies['cast'].apply(lambda x:x[0:3])
movies.head()

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

movies['crew'] = movies['crew'].apply(fetch_director)

movies.head()

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies.sample(5)

movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies.head()

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

movies.head()

new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english') 

vector = cv.fit_transform(new['tags']).toarray()

vector.shape

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

similarity

new[new['title'] == 'The Lego Movie'].index[0]

def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
        

recommend('Gandhi')

import pickle
import gzip
import numpy as np

# Step 1: Reduce precision to 2 decimals (safe for similarity matrix)
similarity = np.round(similarity.astype(np.float16), 2)

# Step 2: Save movie DataFrame
with open("movie_list.pkl", "wb") as f:
    pickle.dump(new, f, protocol=pickle.HIGHEST_PROTOCOL)
print("✅ Movie list saved successfully as movie_list.pkl")

# Step 3: Save similarity matrix with highest pickle protocol
with gzip.open("similarity.pkl.gz", "wb") as f:
    pickle.dump(similarity, f, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ similarity.pkl.gz saved (optimized)")





Gandhi, My Father
The Wind That Shakes the Barley
A Passage to India
Guiana 1838
Ramanujan
✅ Movie list saved successfully as movie_list.pkl
✅ similarity.pkl.gz saved (compressed)
